In [1]:
# Point source matching script between heritage programme and Chandra source catalogue2
# The better PSF of Chandra will enable to accuretly determine the poistion of the source which is necessary for
# finding multiwave length counterpart
# This script creats a source list (cross_match_xmm_chandra.csv) of matched object between XMM and Chandra 
# observations
# AUTHOR : Samaresh Mondal (smondal@camk.edu.pl)
# DATE   : 2021 Sep 06

In [2]:
import numpy as np
import math
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table
from urllib.request import urlopen
from urllib.parse import urlencode
import astropy
import pyvo as vo

In [3]:
cone1 = vo.dal.SCSService('http://cda.cfa.harvard.edu/csc1scs/coneSearch')
cone2 = vo.dal.SCSService('http://cda.cfa.harvard.edu/csc2scs/coneSearch')
maxrad = 15.0 * u.arcsec        # search radius 15 arcsec

In [4]:
# XMM EPIC-PN count rate flux conversion from 0.5-4.5 keV count rate to 0.5-7.0 keV flux
# using nH=1e22, powelaw index=2.0 and normaization=2.212E-03
pn_ctr_to_flux = 4.905e-12
pn_ctr_to_flux1 = 5.111E-12  # this in 0.1 -10 keV band from 0.2-12 keV
m1_ctr_to_flux = 3.738E-12
m2_ctr_to_flux = 3.738E-12

In [5]:
# Observation ids of heritage pointings
obsids_heritage=[['1','0886010101'],['2','0886010401'],['3','0886010501'],
                 ['4','0886010601'],['5','0886010301'],['6','0886010701'],
                 ['7','0886010801'],['8','0886010201'],['9','0886010901'],
                 ['10','0886011001'],['11','0886011101'],['12','0886011201'],
                 ['13','0886020301'],['14','0886020101'],['15','0886020301'],
                 ['16','0886011301']]

In [9]:
fp1 = open("cross_match_xmm_chandra.csv","w")
fp1.write("   %s        %s      %s    %s    %s %s %s %s %s %s\n"%("RA","DEC","XMM-ID","Name","RA(Ch)",\
                                "DEC(Ch)","Sig","XMM-flux(0.2-5keV)","Ch-flux(0.5-7keV)","HR"))
for i in range(len(obsids_heritage)):
    print(i)
    # Reading the xmm source list and matching with chandra source catalogue
    index = i
    obsid = obsids_heritage[index][1]
    xmm = fits.getdata('M1S001M2S002PNS003_'+str(obsid)+'_emllist_formatted.fits', 1)
    txmm = Table(xmm)

    ra_xmm = txmm['RA']                           # in degree
    dec_xmm = txmm['DEC']                         # in degree
    pos_err_xmm = txmm['RADEC_ERR']               # in arcsec
    srcnumber_xmm = txmm['ML_ID_SRC']
    pos_totalerr_xmm = txmm['TOTAL_RADEC_ERR']
 
    # EPIC-PN count rates
    pn_ctr = txmm['RATE_pn']                      # 0.2-12.0 keV
    pn_ctr1 = txmm['RATE_pn_1']                   # 0.2-0.5 keV
    pn_ctr2 = txmm['RATE_pn_2']                   # 0.5-1.0 keV
    pn_ctr3 = txmm['RATE_pn_3']                   # 1.0-2.0 keV
    pn_ctr4 = txmm['RATE_pn_4']                   # 2.0-4.5 keV
    pn_ctr5 = txmm['RATE_pn_5']                   # 4.5-12.0 keV
    pn_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    pn_ctr_err1 = txmm['RATE_ERR_pn_1']
    pn_ctr_err2 = txmm['RATE_ERR_pn_2']
    pn_ctr_err3 = txmm['RATE_ERR_pn_3']
    pn_ctr_err4 = txmm['RATE_ERR_pn_4']
    pn_ctr_err5 = txmm['RATE_ERR_pn_5']
    pn_ctr_err6 = np.sqrt(pow(pn_ctr_err1,2.0)+pow(pn_ctr_err2,2.0)+pow(pn_ctr_err3,2.0)+pow(pn_ctr_err4,2.0))
    #HR_err = 
    
    # EPIC-PN flux 0.5-7 keV band
    pn_flux = (pn_ctr2+pn_ctr3+pn_ctr4)*pn_ctr_to_flux
    
    # PN count rate in full band 0.2-12 keV
    pn_ctr = txmm['RATE_pn']
    pn_ctr_err = txmm['RATE_ERR_pn']
    
    
    # Change the coordinate RA and DEC into a zip
    coo_xmm = SkyCoord(ra_xmm*u.degree, dec_xmm*u.degree)
    
    for j in range(len(coo_xmm)):
        results = cone2.search(pos=coo_xmm[j], radius=maxrad, verbosity=2)  
        # verbosity: column informations 1=minimal
        if(len(results)!=0):
            name = results["name"][0]
            ra_ch = results["ra"][0]
            dec_ch = results["dec"][0]
            sig = results["significance"][0]           # Detectation significence
            #LH = results["likelihood"][0]             # Dtactation likelihood
            #LH_class = results["likelihood_class"][0] # Detectation true, false or marginal
            flux = results["flux_aper_b"][0]           # Broad band 0.5-7 keV
            pn_flux1 = pn_flux[j]                      # 0.5-7.0 keV
            if(math.isnan(flux)==True or flux==0.0):
                flux = results["flux_aper_w"][0]        # Broad band 0.1-10 keV, the source is detected in HRC
                pn_flux1 = pn_ctr[j]*pn_ctr_to_flux1    # 0.1-10 keV band
            fp1.write("%f %f %d %s %f %f %f %g %g %g\n"%(ra_xmm[j],dec_xmm[j],int(obsid),name,ra_ch,\
                                                         dec_ch,sig,pn_flux1,flux,HR[j]))
    
fp1.close()

/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Warning: converting a masked element to nan.
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered 